## Import

In [ ]:
import pandas as pd
import xlrd
import numpy as np
import matplotlib.pyplot as plt
import csv
import scipy.optimize as opt

## File name and experiment set-up

In [ ]:
inhib_file = r'data\MOI 0.01 drug screen 2.xlsx'
tox_file = r'data\Uninfected nCoV drug screen 2.xlsx'
save_folder = r'out'
n_plates = 7
n_columns = 12
n_repeats = 3
n_concentrations = 8
start_concentration = 50

## Names of compounds

In [ ]:
plate = {} # <--- don't change this line!
# format is plate[number]= ['Compound1', 'Compound2', 'Compound 3']
# e.g. plate[1] = ['FD', 'BM', 'AH']
# don't worry about dmso, that's already taken care of :)
plate[1] = ['FD', 'BM', 'AH']
plate[2] = ['CH', 'TC', 'AD']
plate[3] = ['TM', 'MH', 'T']
plate[4] = ['TV', 'A', 'GH']
plate[5] = ['IM', 'F', 'CH']
plate[6] = ['HS', 'PH', 'ED']
plate[7] = ['CP', 'TaC', 'H2O']
#
plate_names = plate

## Prepare array of concentrations

In [ ]:
concentrations = []
concentrations.append(start_concentration)
for i in range(1, n_concentrations):
    concentrations.append(concentrations[i-1]/2)

## Function to read data from excel file into nice dictionary of dataframes

In [ ]:
def get_dict_from_excel(filename, n_plates, n_columns, inhib=True):
    
    if not inhib:
        _n_plates = n_plates+1
    else:
        _n_plates = n_plates
    
    start_identifier = 'Plate:'
    end_identifier = '~End'
    
    book = xlrd.open_workbook(filename=filename)
    sheet = book.sheet_by_index(0)
    row_titles = sheet.col_values(0)
    
    start_indices = []
    end_indices = []
    
    for i in range(len(row_titles)):
        row_title = row_titles[i]
        if row_title==start_identifier:
            start_indices.append(i)
        elif row_title==end_identifier:
            end_indices.append(i)
    
    data = {}
    
    for plate in range(_n_plates):
        
        vals = []
        start = start_indices[plate]+2
        end = end_indices[plate]-1
        
        for col in range(2, 2+n_columns):
            cells = sheet.col_slice(col, start, end)
            for cell in cells:
                vals.append(cell.value)
        
        vals = np.reshape(vals, (n_columns, end-start)).T
        df = pd.DataFrame(data=vals)
        
        for i in range(n_concentrations):
            df.rename(index={i:concentrations[i]}, inplace=True)
           
        if not inhib:
            _plate = plate
        else:
            _plate = plate+1
        
        data[f'Plate {_plate}'] = df
        
    return data

## Function to do averaging of repeats

In [ ]:
def do_repeat_averaging(df, n_repeats):
    
    n_columns = len(df.columns)
    col_names = df.columns
    
    n_samples = int(n_columns/n_repeats)
    
    df_averaged = pd.DataFrame()
       
    for n in range(n_samples):
        i = n*n_repeats
        j = (n+1)*n_repeats-1
        
        drug = col_names[i][:-2]

        _i = col_names[i]
        _j = col_names[j]
        av_col = df.loc[:,_i:_j].T.mean()
        std_col = df.loc[:,_i:_j].T.std()
        df_averaged[f'Mean {drug}'] = av_col
        df_averaged[f'Std {drug}'] = std_col

    return df_averaged

## Functions for toxicity and inhibition calculation

In [ ]:
def get_toxicity(drug, cell, blank=0):
    
    _drug = np.subtract(drug, blank)
    _cell = np.subtract(cell, blank)

    tox = 1 - np.divide(_drug, _cell)
    
    return tox*100

In [ ]:
def get_inhibition(drug, cell, vehicle):
    
    _drug = np.subtract(drug, vehicle)
    _cell = np.subtract(cell, vehicle)

    inhib = np.divide(_drug, _cell)
    
    return inhib*100

In [ ]:
def get_tox_and_inhib_for_plate(inhib_data, tox_data, plate_number, plate_names, n_repeats, concentrations):
    plate_key = f'Plate {plate_number}'
    drug_names = plate_names[plate_number]
    n_drugs = len(drug_names)
    n_concentrations = len(concentrations)
    
    plate_0 = np.asarray(tox_data['Plate 0']).T
    inhib_array = np.asarray(inhib_data[plate_key]).T
    tox_array = np.asarray(tox_data[plate_key]).T
    
    cell = np.mean(plate_0)
    vehicle = np.mean(inhib_array[9:12], axis=0)
    
    df_inhib = pd.DataFrame()
    df_tox = pd.DataFrame()

    
    for d in range(n_drugs):
        drug = drug_names[d]
        
        for r in range(n_repeats):
            i = d*n_repeats + r

            tox = get_toxicity(tox_array[i], cell)
            inhib = get_inhibition(inhib_array[i], cell, vehicle)
            
            df_inhib[f'{drug}-{r+1}'] = inhib
            df_tox[f'{drug}-{r+1}'] = tox
           
    return df_tox, df_inhib        

## Logistic fitting

In [ ]:
def logistic_function(x_vals, top, logEC50, hill_slope):
    x_vals = np.array(x_vals)
    x_vals = np.log10(x_vals)
    bottom = 0
    
    return bottom + (top-bottom)/(1+np.power(10, (logEC50-x_vals)*hill_slope))


def fit_logistic_function(concentrations, responses, verbose=False):
    
    #x_vals = np.log10(concentrations)
    
    params = [100, 0.5, 2]
    
    try:
        popt, pcov = opt.curve_fit(logistic_function, concentrations, responses, p0=params)
    except:
        if verbose:
            print('fitting not possible, sorry!')
        return 0, 0, 0
    
    _top = popt[0]
    _logEC50 = popt[1]
    _hill_slope = popt[2]
    
    _x_vals = np.linspace(concentrations[0], concentrations[-1], 500)
    
    fit = logistic_function(_x_vals, _top, _logEC50, _hill_slope)

    if verbose:
        print(f'Initialised params at: {params}')
        print(f'Optimised params at: {popt}')
        plt.figure()
        plt.plot(concentrations, responses, 'bo', label='raw data')
        plt.plot(_x_vals, fit, 'b', label='fit')
        plt.xscale('log')
    
    return _top, _logEC50, _hill_slope

## Plotting and fitting functions

In [ ]:
def get_arrays(df, _i, _j):
    array = np.asarray(df.loc[:,_i:_j].T)
    mean_array = np.mean(array, axis=0)
    std_array = np.std(array, axis=0)
    array = array.flatten()
    
    return array, mean_array, std_array
        
    
def plots_and_fits(df_tox, df_inhib, concentrations, n_repeats, plate=None, save_path=None):
    n_columns = len(df_tox.columns)
    col_names = df_tox.columns
    
    n_samples = int(n_columns/n_repeats)
    concentrations_array = np.tile(concentrations, n_repeats)
    
    if save_path is not None:
        
        csvfile = open(f'.\{save_path}\Plate_{plate}_fit_params.csv', 'w', newline='')
        csvwriter = csv.writer(csvfile, delimiter=',')
        csvwriter.writerow(['Drug', 'Top_T', 'LogEC50_T', 'EC50_T', 'HillSlope_T', 'Top_I', 'LogEC50_I', 'EC50_I', 'HillSlope_I'])
    
       
    for n in range(n_samples):
        
        i = n*n_repeats
        j = (n+1)*n_repeats-1
        
        drug = col_names[i][:-2]

        _i = col_names[i]
        _j = col_names[j]
        
        i_array, i_mean_array, i_std_array = get_arrays(df_inhib, _i, _j)
        t_array, t_mean_array, t_std_array = get_arrays(df_tox, _i, _j)

        i_t, i_e, i_h = fit_logistic_function(concentrations_array, i_array, verbose=False)
        t_t, t_e, t_h = fit_logistic_function(concentrations_array, t_array, verbose=False)
        
        if save_path is not None:
            row = [drug, t_t, t_e, 10**t_e, t_h, i_t, i_e, 10**i_e, i_h]
            csvwriter.writerow(row)
        
        
        _x = np.linspace(concentrations[0], concentrations[-1], 1000)

        plt.figure(figsize=[5,5])
        
        plt.plot(concentrations_array, i_array, color='skyblue', marker='.', linestyle='')
        plt.errorbar(concentrations, i_mean_array, i_std_array, color='skyblue', marker='o', linestyle='', capsize=3)
        plt.plot(_x, logistic_function(_x, i_t, i_e, i_h), color='dodgerblue')
        
        plt.plot(concentrations_array, t_array, color='coral', marker='.', linestyle='')
        plt.errorbar(concentrations, t_mean_array, t_std_array, color='coral', marker='o', linestyle='', capsize=3)
        plt.plot(_x, logistic_function(_x, t_t, t_e, t_h), color='orangered')
        
        plt.xscale('log')

        plt.ylim([-25, 125])
        plt.xlim([0.1, 100])
        plt.yticks(np.arange(-25, 125, 25),np.arange(-25, 125, 25))
        plt.xticks([0.1, 1, 10, 100], [0.1, 1, 10, 100])
        plt.xlabel('Concentration (uM)')
        plt.ylabel('Percent (%)')
        plt.hlines([0,50,100], 0.1, 100, linestyle=':')
        plt.title(drug)
        
        if save_path is not None:
            plt.savefig(f'.\{save_path}\Plate_{plate}_plot.png', dpi=300)

In [ ]:
inhib_data = get_dict_from_excel(inhib_file, n_plates, n_columns)
tox_data = get_dict_from_excel(tox_file, n_plates, n_columns, inhib=False)

for plate in range(1, n_plates+1):

    df_tox, df_inhib = get_tox_and_inhib_for_plate(inhib_data, tox_data, plate, plate_names, n_repeats, concentrations)

    plots_and_fits(df_tox, df_inhib, concentrations, n_repeats, plate=plate, save_path=save_folder)